# Enable Serverless Compute on Azure Databricks

In this notebook, we will explore **Serverless Compute**, a paradigm shift in how Databricks resources are managed. We will cover what it is, how the architecture differs from classic compute, how to enable it, and how to use it across Notebooks, Jobs, and DLT pipelines.

### Learning Objectives
1.  Understand the concept and benefits of Serverless Compute.
2.  Analyze the **Architecture Shift** (Classic vs. Serverless).
3.  Check Regional Availability.
4.  Enable Serverless Compute in the Account Console.
5.  Set up **Budget Policies** for cost tracking.
6.  Hands-on: Running Serverless Notebooks (Python & SQL).
7.  Hands-on: Using Serverless in Workflows and DLT.

## 1. What is Serverless Compute?

Serverless compute in Databricks means you no longer have to configure, manage, or wait for clusters to start.

*   **Instant Startup:** Serverless compute starts in seconds, unlike classic clusters which can take minutes.
*   **No Administration:** No need to choose instance types, driver sizes, or auto-scaling policies. Databricks manages the fleet.
*   **Elasticity:** It scales up and down instantly based on workload demand.
*   **Cost Efficiency:** You are billed only for the time your code runs (plus a minimal startup time), avoiding idle cluster costs.

### Supported Workloads
As of now, Serverless is available for:
*   Notebooks
*   Jobs / Workflows
*   Delta Live Tables (DLT)
*   Databricks SQL Warehouses

## 2. Architecture Shift: Classic vs. Serverless

There is a fundamental change in where the compute resources live.

### Classic Compute Plane
*   **Location:** Lives in **Your Cloud Account** (the Customer's Data Plane/VNet).
*   **Management:** You manage the VNet, security groups, and quota.
*   **Latency:** VM startup takes time (acquiring resources from Azure/AWS).

### Serverless Compute Plane
*   **Location:** Lives in a **Databricks Managed Compute Plane** (within the Databricks Account).
*   **Connection:** It connects securely to your storage to process data.
*   **Isolation:**
    *   Databricks maintains a warm pool of VMs.
    *   When you request compute, a VM is assigned exclusively to you.
    *   **Security:** Once your task is done, the VM is scrubbed/terminated and never reused for another customer, ensuring strict data isolation.
*   **Networking:** It is typically deployed in the same region as your workspace to minimize latency.

## 3. Prerequisites & Availability

Serverless is not available in every region immediately. Before trying to enable it, you must verify if your workspace region supports it.

1.  Go to the [Azure Databricks Regional Availability](https://learn.microsoft.com/en-us/azure/databricks/resources/feature-region-support) page.
2.  Check the "Serverless SQL and Serverless Compute" column.
3.  Ensure your workspace region (e.g., `Central India`, `East US`) is listed.

*Note: If your workspace is in a region that doesn't support Serverless (e.g., South India at the time of recording), you will not see the option to enable it.*

## 4. How to Enable Serverless

If Serverless is not enabled by default in your workspace (often the case for existing Azure workspaces), an **Account Admin** must enable it.

**Steps:**
1.  Go to **Manage Account** (Account Console).
2.  Navigate to **Settings** -> **Feature Enablement**.
3.  Locate **Serverless compute for Notebooks, Workflows, and Delta Live Tables**.
4.  Toggle the switch to **Enable**.

*Once enabled, the feature becomes available to all eligible workspaces in the account.*

## 5. Cost Management: Budget Policies

Since Serverless resources are managed by Databricks, you pay a "DBU price" that includes both the compute and the software license. To track these costs effectively, we use **Budget Policies**.

### Steps to create a Budget Policy:
1.  Go to Workspace **Settings** -> **Compute**.
2.  Find **Budget Policies** and click **Manage**.
3.  Click **Create Budget Policy**.
    *   **Name:** e.g., `Demo_Budget`
    *   **Tags:** Add custom tags (e.g., `Key: CostCenter`, `Value: DataTeam`).
4.  **Permissions:** By default, only the creator has permission. You must **Grant Access** to other users or groups so they can assign their Serverless workloads to this policy.

*Tagging is crucial for attributing costs in your billing reports.*

## 6. Hands-on: Serverless with Notebooks

Let's see Serverless in action.

**Steps:**
1.  Open a Notebook.
2.  Click the **Connect** dropdown (Compute selector) at the top right.
3.  Select **Serverless**.
4.  Notice the status changes to "Connected" almost instantly.

**Configuration:**
*   You can select **Environment** (Standard vs. High Memory) via the configuration pane.
*   **Languages:** Currently, Serverless for notebooks primarily supports **Python** and **SQL**.
*   **Budget Policy:** You can assign the policy created in step 5 here.

Let's run some code to verify.

In [ ]:
# 1. Run a PySpark command on Serverless
# Note how fast the execution starts compared to waiting for a cluster driver.

display(spark.range(100))

In [ ]:
# 2. Check Spark Configuration context (Optional)
# This confirms we are running in a serverless environment context.

print(spark.conf.get("spark.databricks.service.serverless.enabled", "false"))

In [ ]:
-- 3. Run SQL on Serverless
-- You can use the %sql magic command.

SELECT * FROM range(100);

## 7. Serverless for Jobs & DLT

### Workflows (Jobs)
You can easily swap existing jobs to run on Serverless without rewriting code.
1.  Go to **Workflows**.
2.  Select a Job.
3.  In the **Compute** section (right pane), click **Swap**.
4.  Select **Serverless**.
5.  Update.
*Result:* The job will now spin up instantly and auto-scale.

### Delta Live Tables (DLT)
1.  Go to your DLT Pipeline settings.
2.  Check the box **Serverless**.
3.  Save and Run.
*Benefit:* Serverless DLT allows for faster development cycles and incremental refreshes on materialized views.

## 8. Summary & Key Takeaways

*   **Speed:** Serverless eliminates the "cold start" problem (waiting 5-10 mins for clusters).
*   **Simplicity:** No complex cluster configuration (Node types, Driver sizes).
*   **Cost:** Pay-as-you-go. Auto-termination is aggressive to save costs when idle.
*   **Architecture:** Compute runs in Databricks' secure account, connecting to your storage.
*   **Tracking:** Always use Budget Policies with Tags to monitor Serverless spend.

In the next video, we will dive deeper into **Databricks SQL Warehouses**, which are heavily powered by Serverless technology!